# Groq

[Groq](https://groq.com/) is the AI infrastructure company that delivers fast AI inference. The LPU™ Inference Engine by Groq is a hardware and software platform that delivers exceptional compute speed, quality, and energy efficiency. [Weave](https://weave-docs.wandb.ai/) automatically tracks and logs calls made using Groq chat completion calls.

In [1]:
!pip install -U groq weave -qqq

In order to use Groq, you need to set the `GROQ_API_KEY` environment variable. You can get your API key from [console.groq.com/keys](https://console.groq.com/keys).

In [2]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Tracing

It’s important to store traces of language model applications in a central location, both during development and in production. These traces can be useful for debugging, and as a dataset that will help you improve your application.

Weave will automatically capture traces for [Groq](https://groq.com/). To start tracking, calling `weave.init()` and use the library as normal.

In [ ]:
import weave

weave.init(project_name="quickstart-groq")

In [ ]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

| [![](https://weave-docs.wandb.ai/assets/images/groq_weave_dasboard-5fe948fa979b11cf007af7185a5df660.png)](https://wandb.ai/geekyrakshit/quickstart-groq/weave/calls?filter=%7B%22opVersionRefs%22%3A%5B%22weave%3A%2F%2F%2Fgeekyrakshit%2Fquickstart-groq%2Fop%2Fgroq.chat.completions.create%3A*%22%5D%7D&cols=%7B%22attributes.weave.client_version%22%3Afalse%2C%22attributes.weave.os_name%22%3Afalse%2C%22attributes.weave.os_release%22%3Afalse%2C%22attributes.weave.os_version%22%3Afalse%2C%22attributes.weave.source%22%3Afalse%2C%22attributes.weave.sys_version%22%3Afalse%7D) |
| --- |
| Weave will now track and log all LLM calls made through the Groq library. You can view the traces in the Weave web interface. |

## Track your own ops

Wrapping a function with @weave.op starts capturing inputs, outputs and app logic so you can debug how data flows through your app. You can deeply nest ops and build a tree of functions that you want to track. This also starts automatically versioning code as you experiment to capture ad-hoc details that haven't been committed to git.

Simply create a function decorated with [`@weave.op`](https://weave-docs.wandb.ai/guides/tracking/ops).

In the example below, we have the function `recommend_places_to_visit` which is a function wrapped with `@weave.op` that recommends places to visit in a city.

In [ ]:
import rich


@weave.op()
def recommend_places_to_visit(city: str, model: str="llama3-8b-8192"):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant meant to suggest places to visit in a city",
            },
            {
                "role": "user",
                "content": city,
            }
        ],
        model="llama3-8b-8192",
    )
    return chat_completion.choices[0].message.content


rich.print(recommend_places_to_visit("New York"))
rich.print(recommend_places_to_visit("Paris"))


| [![](https://weave-docs.wandb.ai/assets/images/groq_weave_tracing-74c234466abdc2d58b42ce3f7b1ddc52.png)](https://wandb.ai/geekyrakshit/quickstart-groq/weave/calls?filter=%7B%22opVersionRefs%22%3A%5B%22weave%3A%2F%2F%2Fgeekyrakshit%2Fquickstart-groq%2Fop%2Frecommend_places_to_visit%3A*%22%5D%7D&cols=%7B%22attributes.weave.client_version%22%3Afalse%2C%22attributes.weave.os_name%22%3Afalse%2C%22attributes.weave.os_release%22%3Afalse%2C%22attributes.weave.os_version%22%3Afalse%2C%22attributes.weave.source%22%3Afalse%2C%22attributes.weave.sys_version%22%3Afalse%7D) |
| --- |
| Weave will now track and log all LLM calls made through the Groq library. You can view the traces in the Weave web interface. |

## Create a [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) for easier experimentation

Organizing experimentation is difficult when there are many moving pieces. By using the [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) class, you can capture and organize the experimental details of your app like your system prompt or the model you're using. This helps organize and compare different iterations of your app.

In addition to versioning code and capturing inputs/outputs, a [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) captures structured parameters that control your application’s behavior, making it easy to find what parameters worked best. You can also use Weave a [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) with serve, and [Evaluations](https://weave-docs.wandb.ai/guides/core-types/evaluations).

In the example below, you can experiment with `GroqCityVisitRecommender`. Every time you change one of these, you'll get a new version of `GroqCityVisitRecommender`.

In [ ]:
class GroqCityVisitRecommender(weave.Model):
    model: str
    temperature: float = 0.7

    @weave.op()
    def predict(self, city: str) -> str:
        system_message = {
            "role": "system",
            "content": """
You are a helpful assistant meant to suggest places to visit in a city
""",
        }
        user_message = {"role": "user", "content": city}
        with Groq(api_key=os.environ.get("GROQ_API_KEY")) as groq_client:
            chat_completion = groq_client.chat.completions.create(
                messages=[system_message, user_message],
                model=self.model,
                temperature=self.temperature,
            )
        return chat_completion.choices[0].message.content


city_recommender = GroqCityVisitRecommender(model="llama3-8b-8192")
rich.print(city_recommender.predict("New York"))
rich.print(city_recommender.predict("San Francisco"))


| [![](https://weave-docs.wandb.ai/assets/images/groq_weave_tracing-74c234466abdc2d58b42ce3f7b1ddc52.png)](https://wandb.ai/geekyrakshit/quickstart-groq/weave/calls?filter=%7B%22opVersionRefs%22%3A%5B%22weave%3A%2F%2F%2Fgeekyrakshit%2Fquickstart-groq%2Fop%2FGroqCityVisitRecommender.predict%3A*%22%5D%7D&cols=%7B%22attributes.weave.client_version%22%3Afalse%2C%22attributes.weave.os_name%22%3Afalse%2C%22attributes.weave.os_release%22%3Afalse%2C%22attributes.weave.os_version%22%3Afalse%2C%22attributes.weave.source%22%3Afalse%2C%22attributes.weave.sys_version%22%3Afalse%7D&peekPath=%2Fgeekyrakshit%2Fquickstart-groq%2Fcalls%2F0194cea7-a8e7-76d3-a85d-643695e81474%3Ftracetree%3D1%26feedbackExpand%3D0) |
| --- |
| Tracing and versioning your calls using a [`Model`](https://weave-docs.wandb.ai/guides/core-types/models) |
